# HEC-RAS Pipes, Conduits, and Pump Stations HDF Data Analysis Notebook

This notebook demonstrates how to manipulate and analyze the new HEC-RAS Conduits, Pipes, and Pump Stations results using the ras-commander library. It leverages the HdfPipe and HdfPump classes to streamline data extraction, processing, and visualization.

In [1]:
# Import required Libraries
import subprocess
import sys
import os
from pathlib import Path

def install_module(module_name):
    try:
        __import__(module_name)
    except ImportError:
        print(f"{module_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", module_name])

# List of modules to check and install if necessary
modules = ['h5py', 'numpy', 'requests', 'geopandas', 'matplotlib', 'pandas', 'pyproj', 'shapely', 'xarray']
for module in modules:
    install_module(module)

# Import the rest of the required libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr


In [2]:
# Install ras-commander if you are not in a dev environment. 
# install_module(ras-commander)

## Importing ras-commander flexibly (from package or local dev copy)

In [ ]:
import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import (init_ras_project, HdfBase, HdfUtils, HdfFluvialPluvial, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, HdfResultsXsec, HdfPipe, HdfPump, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras)
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger
except ImportError:
    # If the import fails, add the parent directory to the Python path
    print("Using Local Dev Copy")
    import os
    current_file = Path(os.getcwd()).resolve()
    parent_directory = current_file.parent
    sys.path.append(str(parent_directory))
    
    # Now try to import again
    from ras_commander import (init_ras_project, HdfBase, HdfUtils, HdfFluvialPluvial, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, HdfResultsXsec, HdfPipe, HdfPump, RasExamples, RasCmdr, RasPlan, RasGeo, RasUnsteady, RasUtils, RasPrj, RasGpt, ras)
    from ras_commander.Decorators import standardize_input, log_call
    from ras_commander.LoggingConfig import setup_logging, get_logger

print("ras_commander imported successfully")

In [ ]:
# Download the Pipes Beta project from HEC and run plan 01

# Define the path to the Pipes Beta project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
pipes_ex_path = current_dir / "example_projects" / "Davis"
import logging

# Check if Pipes Beta.p06.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = pipes_ex_path / "DavisStormSystem.p02.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the Pipes Beta project
    ras_examples = RasExamples()
    ras_examples.extract_project(["Davis"])

    # Initialize custom Ras object
    pipes_ex = RasPrj()

    # Initialize the RAS project using the custom ras object
    pipes_ex = init_ras_project(pipes_ex_path, "6.6", ras_instance=pipes_ex)
    logging.info(f"Pipes Beta project initialized with folder: {pipes_ex.project_folder}")
    
    logging.info(f"Pipes Beta object id: {id(pipes_ex)}")
    
    # Define the plan number to execute
    plan_number = "02"

    # Update run flags for the project
    RasPlan.update_run_flags(
        plan_number,
        geometry_preprocessor=True,
        unsteady_flow_simulation=True,
        run_sediment=False,
        post_processor=True,
        floodplain_mapping=False,
        ras_object=pipes_ex
    )

    # Execute Plan 06 using RasCmdr for Pipes Beta
    print(f"Executing Plan {plan_number} for the Pipes Beta Creek project...")
    success_pipes_ex = RasCmdr.compute_plan(plan_number, ras_object=pipes_ex)
    if success_pipes_ex:
        print(f"Plan {plan_number} executed successfully for Pipes Beta.\n")
    else:
        print(f"Plan {plan_number} execution failed for Pipes Beta.\n")
else:
    print("Pipes Beta.p06.hdf already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the custom ras object
    pipes_ex = RasPrj()
    pipes_ex = init_ras_project(pipes_ex_path, "6.6", ras_instance=pipes_ex)
    plan_number = "02"

In [ ]:
# Load Plan and Geometry Dataframes and find Plan and Geometry HDF Paths

# Display plan_df for pipes_ex project
print("Plan DataFrame for pipes_ex project:")
display(pipes_ex.plan_df)

# Display geom_df for pipes_ex project
print("\nGeometry DataFrame for pipes_ex project:")
display(pipes_ex.geom_df)

# Get the plan HDF path
plan_hdf_path = pipes_ex.plan_df.loc[pipes_ex.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]

# Get the geometry file number from the plan DataFrame
geom_file = pipes_ex.plan_df.loc[pipes_ex.plan_df['plan_number'] == plan_number, 'Geom File'].values[0]
geom_number = geom_file[1:]  # Remove the 'g' prefix

# Get the geometry HDF path
geom_hdf_path = pipes_ex.geom_df.loc[pipes_ex.geom_df['geom_number'] == geom_number, 'hdf_path'].values[0]

print(f"\nPlan HDF path for Plan {plan_number}: {plan_hdf_path}")
print(f"Geometry HDF path for Plan {plan_number}: {geom_hdf_path}")

In [ ]:
# Example: Extract runtime and compute time data
print("\nExample 2: Extracting runtime and compute time data")
runtime_df = HdfResultsPlan.get_runtime_data(hdf_input=plan_number, ras_object=pipes_ex)
if runtime_df is not None:
    display(runtime_df)
else:
    print("No runtime data found.")

In [ ]:
# Use explore_hdf5 function to get Pipe Conduits data:
HdfUtils.explore_hdf5(plan_hdf_path, "/Geometry/Pipe Conduits/")

In [ ]:
# Example: Get pipe conduits
pipe_conduits_gdf = HdfPipe.get_pipe_conduits(plan_hdf_path)
print("\nPipe Conduits: pipe_conduits_gdf")
display(pipe_conduits_gdf.head())

pipe_conduits_gdf:  

| Name | System Name | US Node | DS Node | Modeling Approach | Conduit Length | Max Cell Length | Shape | Rise | Span | ... | Slope | US Entrance Loss Coefficient | DS Exit Loss Coefficient | US Backflow Loss Coefficient | DS Backflow Loss Coefficient | DS Flap Gate | Major Group | Minor Group | Polyline | Terrain_Profiles |
|------|-------------|---------|---------|-------------------|----------------|------------------|-------|------|------|-----|-------|------------------------------|--------------------------|------------------------------|------------------------------|--------------|-------------|-------------|----------|-------------------|
| 0    | 134         | Davis   | O13-DMH007 | O13-DMH006 | hydraulic        | 443.740020      | 40.0             | circular | 6.0  | 6.0  | ... | 0.002723 | 0.2                        | 0.4                      | 0.2                          | 0.4                          | 0            | Major Group 2 |             | LINESTRING (6635295.441 1965214.2465, 6635196.... | [(0.0, 40.819695), (21.217846, 40.642994), (35... |
| 1    | 133         | Davis   | O13-DMH024 | O13-DMH009 | hydraulic        | 800.000024      | 40.0             | circular | 6.0  | 6.0  | ... | 0.001904 | 0.2                        | 0.4                      | 0.2                          | 0.4                          | 0            | Major Group 2 |             | LINESTRING (6635295.441 1965214.2465, 6635196.... | [(0.0, 40.530186), (21.1467, 40.44057), (50.88... |
| 2    | 132         | Davis   | O13-DMH006 | O13-SDS03 | hydraulic        | 443.740070      | 40.0             | circular | 6.0  | 6.0  | ... | 0.002816 | 0.2                        | 0.4                      | 0.2                          | 0.4                          | 0            | Major Group 2 |             | LINESTRING (6635295.441 1965214.2465, 6635196.... | [(0.0, 41.700996), (26.817467, 41.552666), (83... |

In [9]:
#!pip install adjusttext #No longer required - optional to help with labels overlapping

In [ ]:
# Plot the pipe conduit linestrings
import matplotlib.pyplot as plt

# Create a new figure with a specified size
plt.figure(figsize=(12, 9))

# Plot each linestring from the GeoDataFrame
for idx, row in pipe_conduits_gdf.iterrows():
    # Extract coordinates from the linestring
    x_coords, y_coords = row['Polyline'].xy
    
    # Plot the linestring
    plt.plot(x_coords, y_coords, 'b-', linewidth=1, alpha=0.7)
    
    # Add vertical line markers at endpoints
    plt.plot([x_coords[0]], [y_coords[0]], 'x', color='black', markersize=4)
    plt.plot([x_coords[-1]], [y_coords[-1]], 'x', color='black', markersize=4)
    
    # Calculate center point of the line
    center_x = (x_coords[0] + x_coords[-1]) / 2
    center_y = (y_coords[0] + y_coords[-1]) / 2
    
    # Add pipe name label at center, oriented top-right
    plt.text(center_x, center_y, f'{row["Name"]}', fontsize=8, 
             verticalalignment='bottom', horizontalalignment='left',
             rotation=45)  # 45 degree angle for top-right orientation

# Add title and labels
plt.title('Pipe Conduit Network Layout')
plt.xlabel('Easting')
plt.ylabel('Northing')

# Add grid
plt.grid(True, linestyle='--', alpha=0.6)

# Adjust layout to prevent label clipping
plt.tight_layout()

# Display the plot
plt.show()


In [ ]:
# Plot the first 2 terrain profiles
import matplotlib.pyplot as plt

# Extract terrain profiles from the GeoDataFrame
terrain_profiles = pipe_conduits_gdf['Terrain_Profiles'].tolist()

# Create separate plots for the first 2 terrain profiles
for i in range(2):
    profile = terrain_profiles[i]
    
    # Unzip the profile into x and y coordinates
    x_coords, y_coords = zip(*profile)
    
    # Create a new figure for each profile
    plt.figure(figsize=(12, 6))
    plt.plot(x_coords, y_coords, marker='o', linestyle='-', color='g', alpha=0.7)
    
    # Add title and labels
    plt.title(f'Terrain Profile {i + 1}')
    plt.xlabel('Distance along profile (m)')
    plt.ylabel('Elevation (m)')
    
    # Add grid
    plt.grid(True, linestyle='--', alpha=0.6)
    
    # Adjust layout to prevent label clipping
    plt.tight_layout()
    
    # Display the plot
    plt.show()


In [12]:
# Use explore_hdf5 function to get Pipe Conduits data:
#HdfUtils.explore_hdf5(plan_hdf_path, "/Geometry/Pipe Nodes/")

In [ ]:
# Example: Get pipe nodes
pipe_nodes_gdf = HdfPipe.get_pipe_nodes(plan_hdf_path)
print("\nPipe Nodes:")
display(pipe_nodes_gdf.head())

pipe_nodes_gdf:


| Name         | System Name | Node Type | Node Status                     | Condtui Connections (US:DS) | Invert Elevation | Base Area | Terrain Elevation | Terrain Elevation Override | Depth     | Drop Inlet Elevation | Drop Inlet Weir Length | Drop Inlet Weir Coefficient | Drop Inlet Orifice Area | Drop Inlet Orifice Coefficient | Total Connection Count | geometry                             |
|--------------|-------------|-----------|----------------------------------|------------------------------|------------------|-----------|-------------------|---------------------------|-----------|----------------------|-------------------------|-----------------------------|-------------------------|-------------------------------|------------------------|-------------------------------------|
| O14-di027   | Davis       | Junction   | Junction with drop inlet        | 1:1                          | 36.060001        | 36.0     | 39.860001         | NaN                       | 3.799999  | 39.863369           | 3.0                     | 3.3                         | 1.0                     | 0.67                          | 2                      | POINT (6637926.81 1964917.32)     |
| P11-DMH004  | Davis       | Junction   | Junction with drop inlet        | 1:1                          | 38.169998        | 36.0     | 48.720001         | NaN                       | 10.550003 | 48.718811           | 3.0                     | 3.3                         | 1.0                     | 0.67                          | 2                      | POINT (6629444.634 1963504.411)   |
| O14-DMH005  | Davis       | Junction   | Junction with drop inlet        | 1:1                          | 31.559999        | 36.0     | 40.840000         | NaN                       | 9.280001  | 40.843731           | 3.0                     | 3.3                         | 1.0                     | 0.67                          | 2                      | POINT (6637368.497 1966084.574)   |

In [14]:
# Use explore_hdf5 function to get Pipe Conduits data:
#HdfUtils.explore_hdf5(plan_hdf_path, "/Geometry/Pipe Networks/")

In [ ]:
# Example: Get pipe network data
pipe_network_gdf = HdfPipe.get_pipe_network(plan_hdf_path)
print("\nPipe Network Data:")
display(pipe_network_gdf.head())


pipe_network_gdf:
| Cell_ID | Conduit_ID | Node_ID | Minimum_Elevation | DS_Face_Indices | Face_Indices | US_Face_Indices | Cell_Property_Info_Index | US Face Elevation | DS Face Elevation | Min Elevation | Area | Info Index | Cell_Polygon | Face_Polylines | Node_Point |
|---------|------------|---------|-------------------|------------------|--------------|------------------|--------------------------|-------------------|-------------------|---------------|------|------------|---------------|----------------|------------|
| 0       | 0          | 0       | -1                | 26.824432        | [1]          | [0, 1]           | [0]                      | 0                 | 26.93429          | 26.824432     | 242.040024 | 0          | POLYGON ((6635288.02154 1965233.24073, 6635279... | [LINESTRING (6635288.021542038 1965233.2407260... | None       |
| 1       | 1          | 0       | -1                | 26.714573        | [2]          | [1, 2]           | [1]                      | 0                 | 26.93429          | 26.824432     | 242.040024 | 0          | POLYGON ((6635288.02154 1965233.24073, 6635279... | [LINESTRING (6635288.021542038 1965233.2407260... | None       |
| 2       | 2          | 0       | -1                | 26.604715        | [3]          | [2, 3]           | [2]                      | 0                 | 26.93429          | 26.824432     | 242.040024 | 0          | POLYGON ((6635288.02154 1965233.24073, 6635279... | [LINESTRING (6635288.021542038 1965233.2407260... | None       |

In [ ]:
# Example: Get pump stations
pump_stations_gdf = HdfPump.get_pump_stations(plan_hdf_path)
print("\nPump Stations:")
display(pump_stations_gdf.head())

Pump Stations:
| geometry                          | station_id | Name             | Inlet River | Inlet Reach | Inlet RS | Inlet RS Distance | Inlet SA/2D | Inlet Pipe Node | Outlet River | ... | Outlet Pipe Node | Reference River | Reference Reach | Reference RS | Reference RS Distance | Reference SA/2D | Reference Point | Reference Pipe Node | Highest Pump Line Elevation | Pump Groups |
|-----------------------------------|------------|------------------|-------------|-------------|----------|-------------------|--------------|------------------|--------------|-----|------------------|------------------|-----------------|--------------|-----------------------|------------------|-----------------|----------------------|------------------------------|-------------|
| POINT (6635027.027 1966080.07)   | 0          | Pump Station #1   |             |             | NaN      |                   |              | Davis [O13-SDS03] |              | ... |                  | NaN              |                 | NaN          |                       | NaN              |                 |                      | NaN                          | 1           |

In [ ]:
# Example: Get pump groups
pump_groups_df = HdfPump.get_pump_groups(plan_hdf_path)
print("\nPump Groups:")
display(pump_groups_df.head())

Pump Groups:
| Pump Station ID | Name             | Bias On | Start Up Time | Shut Down Time | Width | Pumps | efficiency_curve_start | efficiency_curve_count | efficiency_curve |
|------------------|------------------|---------|----------------|----------------|-------|-------|------------------------|-----------------------|------------------|
| 0                | Pump Station #1   | 0       | 5.0            | NaN            | 5.0   | 1     | 0                      | 6                     | [[2.0, 70.0], [4.0, 60.0], [6.0, 55.0], [8.0, ... |

In [ ]:
# Use HdfUtils for extracting projection
print("\nExtracting Projection from HDF")
projection = HdfUtils.projection(hdf_path=geom_hdf_path)
if projection:
    print(f"Projection: {projection}")
else:
    print("No projection information found.")

In [ ]:
# Get projection from HDF file
projection = HdfUtils.projection(hdf_path=geom_hdf_path)

# Set CRS for GeoDataFrames
if projection:
    pipe_conduits_gdf.set_crs(projection, inplace=True, allow_override=True)
    pipe_nodes_gdf.set_crs(projection, inplace=True, allow_override=True)

print("Pipe Conduits GeoDataFrame columns:")
print(pipe_conduits_gdf.columns)

print("\nPipe Nodes GeoDataFrame columns:")
print(pipe_nodes_gdf.columns)

perimeter_polygons = HdfMesh.mesh_areas(geom_hdf_path, ras_object=pipes_ex)
if projection:
    perimeter_polygons.set_crs(projection, inplace=True, allow_override=True)
    
print("\nPerimeter Polygons GeoDataFrame columns:")
print(perimeter_polygons.columns)

In [ ]:
import matplotlib.pyplot as plt
from shapely import wkt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import numpy as np

fig, ax = plt.subplots(figsize=(28, 20))

# Plot cell polygons with 50% transparency behind the pipe network
cell_polygons_df = HdfMesh.mesh_cell_polygons(geom_hdf_path, ras_object=pipes_ex)
if not cell_polygons_df.empty:
    cell_polygons_df.plot(ax=ax, edgecolor='lightgray', facecolor='lightgray', alpha=0.5)

# Plot pipe conduits - the Polyline column already contains LineString geometries
pipe_conduits_gdf.set_geometry('Polyline', inplace=True)

# Plot each pipe conduit individually to ensure all are shown
for idx, row in pipe_conduits_gdf.iterrows():
    ax.plot(*row.Polyline.xy, color='blue', linewidth=1)

# Create a colormap for node elevations
norm = plt.Normalize(pipe_nodes_gdf['Invert Elevation'].min(), 
                    pipe_nodes_gdf['Invert Elevation'].max())
cmap = plt.cm.viridis

# Plot pipe nodes colored by invert elevation
scatter = ax.scatter(pipe_nodes_gdf.geometry.x, pipe_nodes_gdf.geometry.y,
                    c=pipe_nodes_gdf['Invert Elevation'], 
                    cmap=cmap, norm=norm,
                    s=100)

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Invert Elevation (ft)', rotation=270, labelpad=15)

# Add combined labels for invert and drop inlet elevations
for idx, row in pipe_nodes_gdf.iterrows():
    label_text = ""  # Initialize label_text for each node
    # Add drop inlet elevation label if it exists and is not NaN
    if 'Drop Inlet Elevation' in row and not np.isnan(row['Drop Inlet Elevation']):
        label_text += f"TOC: {row['Drop Inlet Elevation']:.2f}\n"
    label_text += f"INV: {row['Invert Elevation']:.2f}"
    
    ax.annotate(label_text,
                xy=(row.geometry.x, row.geometry.y),
                xytext=(-10, -10), textcoords='offset points',
                fontsize=8,
                bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))

# Add perimeter polygons 
if not perimeter_polygons.empty:
    perimeter_polygons.plot(ax=ax, edgecolor='black', facecolor='none')

# Create proxy artists for legend
conduit_line = mlines.Line2D([], [], color='blue', label='Conduits')
node_point = mlines.Line2D([], [], color='blue', marker='o', linestyle='None',
                          markersize=10, label='Nodes')
perimeter = mpatches.Patch(facecolor='none', edgecolor='black',
                          label='Perimeter Polygons')

ax.set_title('Pipe Network with Node Elevations')

# Add legend with proxy artists
ax.legend(handles=[conduit_line, node_point, perimeter])

# Set aspect ratio to be equal and adjust limits
ax.set_aspect('equal', 'datalim')
ax.autoscale_view()

plt.show()


In [ ]:
# Visualize pump stations on a map
fig, ax = plt.subplots(figsize=(12, 8))
pump_stations_gdf.plot(ax=ax, color='green', markersize=50, label='Pump Stations')
ax.set_title('Pump Stations Location')
ax.legend()
plt.show()

In [ ]:
# Example 3: Get pipe network timeseries
valid_variables = [
    "Cell Courant", "Cell Water Surface", "Face Flow", "Face Velocity",
    "Face Water Surface", "Pipes/Pipe Flow DS", "Pipes/Pipe Flow US",
    "Pipes/Vel DS", "Pipes/Vel US", "Nodes/Depth", "Nodes/Drop Inlet Flow",
    "Nodes/Water Surface"
]

print("Valid variables for pipe network timeseries:")
for var in valid_variables:
    print(f"- {var}")

# Extract pipe network timeseries for each valid pipe-related variable
pipe_variables = [var for var in valid_variables if var.startswith("Pipes/") or var.startswith("Nodes/")]

for variable in pipe_variables:
    try:
        pipe_timeseries = HdfPipe.get_pipe_network_timeseries(plan_hdf_path, variable=variable)
        print(f"\nPipe Network Timeseries ({variable}):")
        print(pipe_timeseries.head())  # Print first few rows to avoid overwhelming output
    except Exception as e:
        print(f"Error extracting {variable}: {str(e)}")

### Pipe Network Timeseries Data Description

The `get_pipe_network_timeseries` function returns an xarray DataArray for each variable. Here's a general description of the data structure:

1. **Pipes/Pipe Flow DS and Pipes/Pipe Flow US**:
   - Dimensions: time, location (pipe IDs)
   - Units: ft^3/s (cubic feet per second)
   - Description: Represents the flow rate at the downstream (DS) and upstream (US) ends of pipes over time.

2. **Pipes/Vel DS and Pipes/Vel US**:
   - Dimensions: time, location (pipe IDs)
   - Units: ft/s (feet per second)
   - Description: Shows the velocity at the downstream (DS) and upstream (US) ends of pipes over time.

3. **Nodes/Depth**:
   - Dimensions: time, location (node IDs)
   - Units: ft (feet)
   - Description: Indicates the depth of water at each node over time.

4. **Nodes/Drop Inlet Flow**:
   - Dimensions: time, location (node IDs)
   - Units: cfs (cubic feet per second)
   - Description: Represents the flow rate through drop inlets at each node over time.

5. **Nodes/Water Surface**:
   - Dimensions: time, location (node IDs)
   - Units: ft (feet)
   - Description: Shows the water surface elevation at each node over time.

General notes:
- The 'time' dimension represents the simulation timesteps.
- The 'location' dimension represents either pipe IDs or node IDs, depending on the variable.
- The number of timesteps and locations may vary depending on the specific dataset and simulation setup.
- Negative values in flow variables may indicate reverse flow direction.



In [ ]:
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import random

# Define the variables we want to plot
variables = [
    "Pipes/Pipe Flow DS", "Pipes/Pipe Flow US", "Pipes/Vel DS", "Pipes/Vel US",
    "Nodes/Depth", "Nodes/Drop Inlet Flow", "Nodes/Water Surface"
]

# Create a separate plot for each variable
for variable in variables:
    try:
        # Get the data for the current variable
        data = HdfPipe.get_pipe_network_timeseries(plan_hdf_path, variable=variable)
        
        # Create a new figure
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Pick one random location
        random_location = random.choice(data.location.values)
        
        # Determine if it's a pipe or node variable
        if variable.startswith("Pipes/"):
            location_type = "Conduit ID"
        else:
            location_type = "Node ID"
        
        # Plot the data for the randomly selected location
        ax.plot(data.time, data.sel(location=random_location), label=f'{location_type} {random_location}')
        
        # Set the title and labels
        ax.set_title(f'{variable} Over Time ({location_type} {random_location})')
        ax.set_xlabel('Time')  # Corrected from ax.xlabel to ax.set_xlabel
        ax.set_ylabel(f'{variable} ({data.attrs["units"]})')  # Corrected from ax.ylabel to ax.set_ylabel
        
        # Format the x-axis to show dates nicely
        ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d %H:%M'))
        plt.xticks(rotation=45)
        
        # Add a legend
        ax.legend(title=location_type, loc='upper left')
        
        # Adjust the layout
        plt.tight_layout()
        
        # Show the plot
        plt.show()
        
    except Exception as e:
        print(f"Error plotting {variable}: {str(e)}")


In [ ]:
# Example 8: Get pump station timeseries
pump_station_name = pump_stations_gdf.iloc[0]['Name']  # Get the first pump station name
pump_timeseries = HdfPump.get_pump_station_timeseries(plan_hdf_path, pump_station=pump_station_name)
print(f"\nPump Station Timeseries ({pump_station_name}):")
print(pump_timeseries)

In [ ]:
# Use explore_hdf5 function to get Pipe Conduits data:
HdfUtils.explore_hdf5(plan_hdf_path, "/Geometry/Pump Stations/")

In [ ]:
# Extract the pump station timeseries data
pump_station_name = pump_stations_gdf.iloc[0]['Name']  # Get the first pump station name
pump_timeseries = HdfPump.get_pump_station_timeseries(plan_hdf_path, pump_station=pump_station_name)

# Print the pump station timeseries
print(f"\nPump Station Timeseries ({pump_station_name}):")
print(pump_timeseries)

# Create a new figure for plotting
fig, ax = plt.subplots(figsize=(12, 12))

# Plot each variable in the timeseries
for variable in pump_timeseries.coords['variable'].values:
    data = pump_timeseries.sel(variable=variable)
    
    # Decode units to strings
    unit = pump_timeseries.attrs["units"][list(pump_timeseries.coords["variable"].values).index(variable)][1].decode('utf-8')
    
    # Check if the variable is 'Pumps on' to plot it differently
    if variable == 'Pumps on':
        # Plot with color based on the on/off status
        colors = ['green' if val > 0 else 'red' for val in data.values.flatten()]
        ax.scatter(pump_timeseries['time'], data, label=f'{variable} ({unit})', color=colors)
    else:
        ax.plot(pump_timeseries['time'], data, label=f'{variable} ({unit})')
        
        # Label the peak values
        peak_time = pump_timeseries['time'][data.argmax()]
        peak_value = data.max()
        ax.annotate(f'Peak: {peak_value:.2f}', xy=(peak_time, peak_value), 
                    xytext=(peak_time, peak_value + 0.1 * peak_value), 
                    arrowprops=dict(facecolor='black', arrowstyle='->'),
                    fontsize=10, color='black', ha='center')

# Set the title and labels
ax.set_title(f'Timeseries Data for Pump Station: {pump_station_name}')
ax.set_xlabel('Time')
ax.set_ylabel('Values')

# Format the x-axis to show dates nicely
ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d %H:%M'))
plt.xticks(rotation=45)

# Add a legend
ax.legend(title='Variables', loc='upper left')

# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()
